In [6]:
import pandas as pd

# 파일 읽기
a_df = pd.read_csv('A.csv')
b_df = pd.read_csv('B.csv')

# 'outer' 방식으로 머지하여 A.csv와 B.csv의 모든 정보를 포함
merged_df = a_df.merge(b_df[['content_url', 'view_count', 'content_type', 'content_id']], on='content_url', how='outer')

# view_count 컬럼의 값을 새로운 컬럼 'view_count(0925)'에 복사
merged_df['view_count(1011)'] = merged_df['view_count']

# 불필요한 view_count 컬럼 삭제
merged_df.drop(columns='view_count', inplace=True)

# 463 행부터 시작하여 새로운 정보를 추가
#new_data = b_df[~b_df['content_url'].isin(a_df['content_url'])]
#merged_df = pd.concat([merged_df.iloc[:463], new_data, merged_df.iloc[463:]], sort=False).reset_index(drop=True)

# 결과 저장
merged_df.to_csv('A_B_merged_1011.csv', index=False)


In [16]:
import pandas as pd

# 파일 읽기
a_df = pd.read_csv('A.csv')
b_df = pd.read_csv('B.csv')

# 'outer' 방식으로 머지하여 A.csv와 B.csv의 모든 정보를 포함
merged_df = a_df.merge(b_df[['content_url', 'view_count', 'content_type', 'content_id']], on='content_url', how='outer')

print(b_df.columns)
print(a_df.columns)

# view_count 컬럼의 값을 새로운 컬럼 'view_count(0925)'에 복사
merged_df['view_count(0925)'] = merged_df['view_count']

# 불필요한 view_count 컬럼 삭제
merged_df.drop(columns='view_count', inplace=True)

def generate_gif_link(url):
    # URL이 NaN 또는 null인 경우, 빈 문자열 반환
    if pd.isnull(url):
        return ""
    gif_id = url.split("/")[-1]
    gif_link = f'<iframe src="https://giphy.com/embed/{gif_id}" width="64" height="64" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="{url}">via GIPHY</a></p>'
    return gif_link

merged_df['gif_link'] = merged_df['content_url'].apply(generate_gif_link)

# 463 행부터 시작하여 새로운 정보를 추가
new_data = b_df[~b_df['content_url'].isin(a_df['content_url'])]
merged_df = pd.concat([merged_df.iloc[:463], new_data, merged_df.iloc[463:]], sort=False).reset_index(drop=True)

# HTML 파일 작성
html_start = """
<html>
<head>
    <title>GIFs from Merged CSV</title>
    <style>
        table {
            border-collapse: collapse;
            width: 100%;
        }

        th, td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
    </style>
</head>
<body>
<table>
    <tr>
        <th>Content URL</th>
        <th>Content ID</th>
        <th>GIF</th>
    </tr>
"""

html_end = """
</table>
</body>
</html>
"""

# 각 행에 대해 HTML 행을 생성
rows = []
for _, row in merged_df.iterrows():
    rows.append(f"""
    <tr>
        <td>{row['content_url']}</td>
        <td>{row['content_id']}</td>
        <td>{row['gif_link']}</td>
    </tr>
    """)

# HTML 코드 결합
full_html = html_start + "\n".join(rows) + html_end

# HTML 파일로 저장
with open("gifs.html", "w", encoding="utf-8") as f:
    f.write(full_html)

# 결과 저장
merged_df.to_csv('A_B_merged.csv', index=False)


Index(['content_url', 'view_count', 'content_type', 'content_id'], dtype='object')
Index(['content_url', 'view_count(0906)', 'view_count(0907)',
       'view_count(0908)', 'view_count(0925)', 'amount of change',
       'content_type', 'content_id'],
      dtype='object')


In [26]:
import pandas as pd

# A_B_merged.csv 파일 읽기
merged_df = pd.read_csv('A_B_merged.csv')

# DataFrame을 HTML로 변환
html_content = merged_df.to_html(escape=False)  # escape=False는 HTML 태그를 올바르게 표현하기 위함입니다.

# HTML 파일로 저장
with open('A_B_merged.html', 'w', encoding='utf-8') as f:
    f.write(html_content)



In [ ]:
[Sort기능이 있는 HTML 만들기]

In [21]:
import pandas as pd

# A_B_merged.csv 파일 읽기
merged_df = pd.read_csv('A_B_merged.csv')

# DataFrame을 HTML로 변환
html_content = merged_df.to_html(escape=False, classes="dataTable", index=False)

# HTML 헤더와 DataTables 스크립트 추가
html_complete = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>A_B_merged</title>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.11.4/css/jquery.dataTables.css">
    <script type="text/javascript" charset="utf8" src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.11.4/js/jquery.dataTables.js"></script>
</head>
<body>
    {html_content}
    <script>
        $(document).ready( function () {{
            $('.dataTable').DataTable();
        }});
    </script>
</body>
</html>
"""

# HTML 파일로 저장
with open('A_B_merged_interactive.html', 'w', encoding='utf-8') as f:
    f.write(html_complete)


In [ ]:
[A_B_merged.csv 파일을 HTML로 변환 시 변화량에 대한 DIFF를 Graph로 변환하는 코드]

In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import base64
from io import BytesIO

def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=80)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("utf-8")

merged_df = pd.read_csv('A_B_merged.csv')

columns = ['view_count(0906)', 'view_count(0907)', 'view_count(0908)', 'view_count(0925)', 'view_count(0926)']

graph_images = []

y_min = merged_df[columns].diff(axis=1).min().min()
y_max = merged_df[columns].diff(axis=1).max().max()

for idx, row in merged_df.iterrows():
    data = row[columns].diff()
    fig, ax = plt.subplots(figsize=(4.5, 3))
    data.plot(kind="line", ax=ax)
    ax.set_ylim(y_min, y_max)  # Y축 스케일 동일하게 설정

    # NaN 또는 무한대 값이 있는 경우 0으로 대체
    data = data.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    ax.set_xticks(range(len(data.index)))
    ax.set_xticklabels(data.index, rotation=45, ha="right")  # X축 라벨 45도 회전
    
    # X축 인덱스 값 및 변화량 표시
    #differences = data.diff().fillna(0)
    #for i, (index, value) in enumerate(data.items()):
    #    ax.text(i, value, f"{value} (+{differences[index]})", ha='center', va='bottom')
    
    
    plt.tight_layout()
    plt.close(fig)
    
    graph_images.append(f"<img src='data:image/png;base64,{fig_to_base64(fig)}' />")

merged_df['Graph'] = graph_images

# Convert to HTML
html_string = merged_df.to_html(escape=False, index=False)
with open("A_B_merged.html", "w", encoding="utf-8") as f:
    f.write(html_string)


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO

def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=80)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("utf-8")

merged_df = pd.read_csv('A_B_merged.csv')
columns = ['view_count(0906)', 'view_count(0907)', 'view_count(0908)', 'view_count(0925)', 'view_count(0926)']

graph_images = []

y_min = merged_df[columns].diff(axis=1).min().min()
y_max = merged_df[columns].diff(axis=1).max().max()
total = len(merged_df)

for idx, row in merged_df.iterrows():
    progress = int((idx / total) * 10)
    bar = "|" + "-" * progress + ">" + " " * (10 - progress) + f"| {progress * 10}%"
    print(bar, end="\r")
    
    data = row[columns].diff()
    fig, ax = plt.subplots(figsize=(4.5, 3))
    data.plot(kind="line", ax=ax)
    ax.set_ylim(y_min, y_max)
    data = data.replace([np.inf, -np.inf], np.nan).fillna(0)
    ax.set_xticks(range(len(data.index)))
    ax.set_xticklabels(data.index, rotation=45, ha="right")
    plt.tight_layout()
    plt.close(fig)
    graph_images.append(f"<img class='graphImg' src='data:image/png;base64,{fig_to_base64(fig)}' />")

merged_df['Graph'] = graph_images

# Convert to HTML with toggle button
html_code = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script>
        $(document).ready(function() {{
            $("#toggleGIF").click(function() {{
                $(".graphImg").toggle();
            }});
        }});
    </script>
</head>
<body>
    <button id="toggleGIF">Toggle GIFs</button>
    {merged_df.to_html(escape=False, index=False)}
</body>
</html>
"""

with open("A_B_merged.html", "w", encoding="utf-8") as f:
    f.write(html_code)


In [11]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO
from tqdm import tqdm

def fig_to_base64(fig):
    buf = BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight", dpi=80)
    buf.seek(0)
    return base64.b64encode(buf.read()).decode("utf-8")

merged_df = pd.read_csv('A_B_merged_1011.csv')
columns = ['view_count(0906)', 'view_count(0907)', 'view_count(0908)', 'view_count(0925)', 'view_count(0926)','view_count(1005)','view_count(1006)','view_count(1010)','view_count(1011)']

graph_images = []

y_min = merged_df[columns].diff(axis=1).min().min()
y_max = merged_df[columns].diff(axis=1).max().max()

# tqdm 라이브러리를 사용하여 진행 상황을 바타입으로 표시합니다.
for idx, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0], desc="Generating Graphs"):
    data = row[columns].diff()
    fig, ax = plt.subplots(figsize=(4.5, 3))
    data.plot(kind="line", ax=ax)
    ax.set_ylim(y_min, y_max)
    data = data.replace([np.inf, -np.inf], np.nan).fillna(0)
    ax.set_xticks(range(len(data.index)))
    ax.set_xticklabels(data.index, rotation=45, ha="right")
    plt.tight_layout()
    plt.close(fig)
    graph_images.append(f"<img class='graphImg' src='data:image/png;base64,{fig_to_base64(fig)}' />")

merged_df['Graph'] = graph_images

# Convert to HTML with toggle button
html_code = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script>
        $(document).ready(function() {{
            $("#toggleGIF").click(function() {{
                $(".graphImg").toggle();
            }});
        }});
    </script>
</head>
<body>
    <button id="toggleGIF">Toggle GIFs</button>
    {merged_df.to_html(escape=False, index=False)}
</body>
</html>
"""

with open("A_B_merged_1011.html", "w", encoding="utf-8") as f:
    f.write(html_code)


Generating Graphs: 100%|█████████████████████████████████████████████████████████████| 472/472 [01:02<00:00,  7.60it/s]
